In [1]:
# Import required packages
import os
import shutil

import keras.callbacks as kc
import keras.layers as kl
import keras.models as km
import keras.optimizers as ko
import keras.utils as ku
import numpy as np
import sklearn as sk
import sklearn.utils as sku

import Config as conf
import CSV as csv

Using TensorFlow backend.


In [2]:
# Check output directory
outputDir = conf.OUTPUT_PATH.format(conf.LEARNING_RATE, conf.BATCH_SIZE,
                                    conf.N_HIDDEN, "")
if os.path.exists(outputDir):
    shutil.rmtree(outputDir)
os.makedirs(outputDir)
logDir = conf.LOG_PATH.format(conf.LEARNING_RATE, conf.BATCH_SIZE,
                              conf.N_HIDDEN, "")
if os.path.exists(logDir):
    shutil.rmtree(logDir)
os.makedirs(logDir)

In [3]:
# Setup Keras Callbacks
tensorboard = kc.TensorBoard(
    log_dir=logDir,
    histogram_freq=0,
    batch_size=conf.BATCH_SIZE,
    write_graph=True,
    write_grads=True,
    write_images=True,
    update_freq=10)
print(
    "If you have tensorboard in this environment, you can type below to see the result in tensorboard:"
)
print("    tensorboard --logdir=" + logDir)

If you have tensorboard in this environment, you can type below to see the result in tensorboard:
    tensorboard --logdir=./Log_LR0.0001_BATCH64_HIDDEN540/


In [4]:
# Notice Checkpoint Directory
print("Keras checkpoints and final result will be saved in here:")
print("    " + outputDir)

Keras checkpoints and final result will be saved in here:
    ./Output_LR0.0001_BATCH64_HIDDEN540/


In [15]:
# Setup Keras RNN Model
lstm = kl.LSTM(
    conf.N_HIDDEN,
    unit_forget_bias=True,
    input_shape=(conf.WINDOW_SIZE, conf.N_INPUT))
lstm.add_loss(1e-8)
adam = ko.Adam(lr=conf.LEARNING_RATE, amsgrad=True)
model = km.Sequential()
model.add(lstm)
model.add(kl.Dense(
    conf.USE_NOACTIVITY and conf.N_CLASSES or conf.N_VALID_CLASSES, activation="softmax"))
model.compile(
    loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

In [ ]:
# Import CSV data
xs, ys = csv.getCSV()

Input directory not found. Calculate merged CSVs...
About enh1 ...
Processing X:  ./Dataset\csi_enh1_1.csv


In [7]:
# Shuffle data
for a in conf.ACTIONS:
    xs[a], ys[a] = sku.shuffle(xs[a], ys[a], random_state=0)

In [16]:
# Run KFold
for i in range(conf.KFOLD):
    # Roll the data
    for a in conf.ACTIONS:
        xs[a] = np.roll(xs[a], int(len(xs[a]) / conf.KFOLD), axis=0)
        ys[a] = np.roll(ys[a], int(len(ys[a]) / conf.KFOLD), axis=0)

    # Data separation
    xTrain = []
    yTrain = []
    xEval = []
    yEval = []
    for a in conf.ACTIONS:
        if xTrain == []:
            xTrain = xs[a][int(len(xs[a]) / conf.KFOLD):]
        else:
            xTrain = np.r_[xTrain, xs[a][int(len(xs[a]) / conf.KFOLD):]]
        if yTrain == []:
            yTrain = ys[a][int(len(ys[a]) / conf.KFOLD):]
        else:
            yTrain = np.r_[yTrain, ys[a][int(len(ys[a]) / conf.KFOLD):]]
        if xEval == []:
            xEval = xs[a][:int(len(xs[a]) / conf.KFOLD)]
        else:
            xEval = np.r_[xEval, xs[a][:int(len(xs[a]) / conf.KFOLD)]]
        if yEval == []:
            yEval = ys[a][:int(len(ys[a]) / conf.KFOLD)]
        else:
            yEval = np.r_[yEval, ys[a][:int(len(ys[a]) / conf.KFOLD)]]

    if not conf.USE_NOACTIVITY:
        # Remove NoActivity from ys
        yTrain = yTrain[:, 1:]
        yEval = yEval[:, 1:]

        # If there exists only one action, convert Y to binary form
        if yEval.shape[1] == 1:
            yTrain = ku.to_categorical(yTrain)
            yEval = ku.to_categorical(yEval)

    # Setup Keras Checkpoint
    checkpoint = kc.ModelCheckpoint(
        outputDir + "Checkpoint_K" + str(i) + "_EPOCH{epoch}_ACC{val_acc:.4f}.h5", period=25)

    # Fit model (learn)
    print(
        str(i + 1) + " th fitting started. Endpoint is " + str(conf.KFOLD) +
        " th.")

    model.fit(
        xTrain,
        yTrain,
        batch_size=conf.BATCH_SIZE,
        epochs=conf.N_ITERATIONS,
        verbose=1,
        callbacks=[tensorboard, checkpoint],
        validation_data=(xEval, yEval))  #, validation_freq=2)
print("Epoch completed!")

C:\Users\chish\.conda\envs\ppp\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  
C:\Users\chish\.conda\envs\ppp\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
C:\Users\chish\.conda\envs\ppp\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
C:\Users\chish\.conda\envs\ppp\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


0 th fitting started. Endpoint is 5 th.
Instructions for updating:
Use tf.cast instead.
Train on 652 samples, validate on 161 samples
Epoch 1/512
652/652 [==============================] - 23s 35ms/step - loss: 0.7270 - acc: 0.6979 - val_loss: 0.4569 - val_acc: 0.8447
Epoch 2/512
652/652 [==============================] - 19s 29ms/step - loss: 0.3768 - acc: 0.8681 - val_loss: 0.2916 - val_acc: 0.8944
Epoch 3/512
652/652 [==============================] - 18s 28ms/step - loss: 0.2566 - acc: 0.9356 - val_loss: 0.2154 - val_acc: 0.9317
Epoch 4/512
652/652 [==============================] - 17s 26ms/step - loss: 0.1974 - acc: 0.9601 - val_loss: 0.1732 - val_acc: 0.9441
Epoch 5/512
652/652 [==============================] - 18s 27ms/step - loss: 0.1520 - acc: 0.9617 - val_loss: 0.1343 - val_acc: 0.9752
Epoch 6/512
652/652 [==============================] - 18s 28ms/step - loss: 0.1312 - acc: 0.9770 - val_loss: 0.1368 - val_acc: 0.9627
Epoch 7/512
652/652 [==============================] - 1

652/652 [==============================] - 19s 29ms/step - loss: 8.0199e-04 - acc: 1.0000 - val_loss: 8.5061e-04 - val_acc: 1.0000
Epoch 59/512
652/652 [==============================] - 20s 30ms/step - loss: 7.2085e-04 - acc: 1.0000 - val_loss: 8.2589e-04 - val_acc: 1.0000
Epoch 60/512
652/652 [==============================] - 20s 30ms/step - loss: 6.6877e-04 - acc: 1.0000 - val_loss: 7.3791e-04 - val_acc: 1.0000
Epoch 61/512
652/652 [==============================] - 19s 28ms/step - loss: 6.2036e-04 - acc: 1.0000 - val_loss: 6.8104e-04 - val_acc: 1.0000
Epoch 62/512
652/652 [==============================] - 20s 30ms/step - loss: 5.8178e-04 - acc: 1.0000 - val_loss: 6.5342e-04 - val_acc: 1.0000
Epoch 63/512
652/652 [==============================] - 19s 29ms/step - loss: 5.4428e-04 - acc: 1.0000 - val_loss: 6.1666e-04 - val_acc: 1.0000
Epoch 64/512
652/652 [==============================] - 18s 28ms/step - loss: 5.1645e-04 - acc: 1.0000 - val_loss: 5.8844e-04 - val_acc: 1.0000
Epoch

Epoch 115/512
652/652 [==============================] - 18s 28ms/step - loss: 1.2887e-04 - acc: 1.0000 - val_loss: 1.6625e-04 - val_acc: 1.0000
Epoch 116/512
652/652 [==============================] - 17s 27ms/step - loss: 1.2573e-04 - acc: 1.0000 - val_loss: 1.6261e-04 - val_acc: 1.0000
Epoch 117/512
652/652 [==============================] - 18s 27ms/step - loss: 1.2185e-04 - acc: 1.0000 - val_loss: 1.5705e-04 - val_acc: 1.0000
Epoch 118/512
652/652 [==============================] - 18s 28ms/step - loss: 1.1673e-04 - acc: 1.0000 - val_loss: 1.5184e-04 - val_acc: 1.0000
Epoch 119/512
652/652 [==============================] - 18s 28ms/step - loss: 1.1395e-04 - acc: 1.0000 - val_loss: 1.5455e-04 - val_acc: 1.0000
Epoch 120/512
652/652 [==============================] - 18s 28ms/step - loss: 1.1200e-04 - acc: 1.0000 - val_loss: 1.5120e-04 - val_acc: 1.0000
Epoch 121/512
652/652 [==============================] - 20s 31ms/step - loss: 1.1077e-04 - acc: 1.0000 - val_loss: 1.4877e-04 - v

652/652 [==============================] - 23s 35ms/step - loss: 5.8439e-05 - acc: 1.0000 - val_loss: 1.0608e-04 - val_acc: 1.0000
Epoch 172/512
652/652 [==============================] - 19s 30ms/step - loss: 5.7804e-05 - acc: 1.0000 - val_loss: 1.0503e-04 - val_acc: 1.0000
Epoch 173/512
652/652 [==============================] - 18s 27ms/step - loss: 5.7177e-05 - acc: 1.0000 - val_loss: 1.0349e-04 - val_acc: 1.0000
Epoch 174/512
652/652 [==============================] - 20s 30ms/step - loss: 5.6484e-05 - acc: 1.0000 - val_loss: 1.0291e-04 - val_acc: 1.0000
Epoch 175/512
652/652 [==============================] - 19s 29ms/step - loss: 5.5821e-05 - acc: 1.0000 - val_loss: 1.0247e-04 - val_acc: 1.0000
Epoch 176/512
652/652 [==============================] - 19s 28ms/step - loss: 5.5142e-05 - acc: 1.0000 - val_loss: 7.5202e-05 - val_acc: 1.0000
Epoch 177/512
652/652 [==============================] - 19s 29ms/step - loss: 5.4437e-05 - acc: 1.0000 - val_loss: 1.0069e-04 - val_acc: 1.0000

...

652/652 [==============================] - 19s 30ms/step - loss: 1.9709e-05 - acc: 1.0000 - val_loss: 1.7632e-05 - val_acc: 1.0000
Epoch 258/512
652/652 [==============================] - 21s 32ms/step - loss: 1.9698e-05 - acc: 1.0000 - val_loss: 1.7620e-05 - val_acc: 1.0000
Epoch 259/512
652/652 [==============================] - 19s 30ms/step - loss: 1.9687e-05 - acc: 1.0000 - val_loss: 1.7615e-05 - val_acc: 1.0000
Epoch 260/512
652/652 [==============================] - 21s 32ms/step - loss: 1.9676e-05 - acc: 1.0000 - val_loss: 1.7610e-05 - val_acc: 1.0000
Epoch 261/512
652/652 [==============================] - 19s 29ms/step - loss: 1.9633e-05 - acc: 1.0000 - val_loss: 1.7600e-05 - val_acc: 1.0000
Epoch 262/512
652/652 [==============================] - 20s 30ms/step - loss: 1.9621e-05 - acc: 1.0000 - val_loss: 1.7590e-05 - val_acc: 1.0000
Epoch 263/512
652/652 [==============================] - 20s 31ms/step - loss: 1.9610e-05 - acc: 1.0000 - val_loss: 1.7583e-05 - val_acc: 1.0000

652/652 [==============================] - 25s 38ms/step - loss: 1.9103e-05 - acc: 1.0000 - val_loss: 1.7176e-05 - val_acc: 1.0000
Epoch 314/512
652/652 [==============================] - 21s 32ms/step - loss: 1.9095e-05 - acc: 1.0000 - val_loss: 1.7170e-05 - val_acc: 1.0000
Epoch 315/512
652/652 [==============================] - 20s 30ms/step - loss: 1.9085e-05 - acc: 1.0000 - val_loss: 1.7153e-05 - val_acc: 1.0000
Epoch 316/512
652/652 [==============================] - 21s 32ms/step - loss: 1.9075e-05 - acc: 1.0000 - val_loss: 1.7148e-05 - val_acc: 1.0000
Epoch 317/512
652/652 [==============================] - 23s 35ms/step - loss: 1.9066e-05 - acc: 1.0000 - val_loss: 1.7138e-05 - val_acc: 1.0000
Epoch 318/512
652/652 [==============================] - 21s 33ms/step - loss: 1.9056e-05 - acc: 1.0000 - val_loss: 1.7137e-05 - val_acc: 1.0000
Epoch 319/512
652/652 [==============================] - 20s 31ms/step - loss: 1.9046e-05 - acc: 1.0000 - val_loss: 1.7129e-05 - val_acc: 1.0000

652/652 [==============================] - 22s 34ms/step - loss: 1.8591e-05 - acc: 1.0000 - val_loss: 1.6741e-05 - val_acc: 1.0000
Epoch 370/512
652/652 [==============================] - 22s 33ms/step - loss: 1.8582e-05 - acc: 1.0000 - val_loss: 1.6730e-05 - val_acc: 1.0000
Epoch 371/512
652/652 [==============================] - 22s 34ms/step - loss: 1.8574e-05 - acc: 1.0000 - val_loss: 1.6719e-05 - val_acc: 1.0000
Epoch 372/512
652/652 [==============================] - 24s 36ms/step - loss: 1.8564e-05 - acc: 1.0000 - val_loss: 1.6707e-05 - val_acc: 1.0000
Epoch 373/512
652/652 [==============================] - 24s 37ms/step - loss: 1.8555e-05 - acc: 1.0000 - val_loss: 1.6700e-05 - val_acc: 1.0000
Epoch 374/512
652/652 [==============================] - 22s 34ms/step - loss: 1.8546e-05 - acc: 1.0000 - val_loss: 1.6693e-05 - val_acc: 1.0000
Epoch 375/512
652/652 [==============================] - 22s 34ms/step - loss: 1.8537e-05 - acc: 1.0000 - val_loss: 1.6685e-05 - val_acc: 1.0000

652/652 [==============================] - 31s 48ms/step - loss: 1.8057e-05 - acc: 1.0000 - val_loss: 1.6297e-05 - val_acc: 1.0000
Epoch 426/512
652/652 [==============================] - 31s 48ms/step - loss: 1.8045e-05 - acc: 1.0000 - val_loss: 1.6289e-05 - val_acc: 1.0000
Epoch 427/512
652/652 [==============================] - 21s 32ms/step - loss: 1.8034e-05 - acc: 1.0000 - val_loss: 1.6282e-05 - val_acc: 1.0000
Epoch 428/512
652/652 [==============================] - 23s 36ms/step - loss: 1.8021e-05 - acc: 1.0000 - val_loss: 1.6271e-05 - val_acc: 1.0000
Epoch 429/512
652/652 [==============================] - 22s 34ms/step - loss: 1.8009e-05 - acc: 1.0000 - val_loss: 1.6264e-05 - val_acc: 1.0000
Epoch 430/512
652/652 [==============================] - 21s 33ms/step - loss: 1.7996e-05 - acc: 1.0000 - val_loss: 1.6256e-05 - val_acc: 1.0000
Epoch 431/512
652/652 [==============================] - 22s 34ms/step - loss: 1.7984e-05 - acc: 1.0000 - val_loss: 1.6248e-05 - val_acc: 1.0000

652/652 [==============================] - 20s 31ms/step - loss: 1.6870e-05 - acc: 1.0000 - val_loss: 1.5089e-05 - val_acc: 1.0000
Epoch 482/512
652/652 [==============================] - 24s 38ms/step - loss: 1.6850e-05 - acc: 1.0000 - val_loss: 1.5058e-05 - val_acc: 1.0000
Epoch 483/512
652/652 [==============================] - 21s 32ms/step - loss: 1.6828e-05 - acc: 1.0000 - val_loss: 1.5037e-05 - val_acc: 1.0000
Epoch 484/512
652/652 [==============================] - 22s 34ms/step - loss: 1.6807e-05 - acc: 1.0000 - val_loss: 1.5008e-05 - val_acc: 1.0000
Epoch 485/512
652/652 [==============================] - 20s 31ms/step - loss: 1.6797e-05 - acc: 1.0000 - val_loss: 1.4981e-05 - val_acc: 1.0000
Epoch 486/512
652/652 [==============================] - 23s 35ms/step - loss: 1.6773e-05 - acc: 1.0000 - val_loss: 1.4984e-05 - val_acc: 1.0000
Epoch 487/512
652/652 [==============================] - 21s 32ms/step - loss: 1.6756e-05 - acc: 1.0000 - val_loss: 1.4968e-05 - val_acc: 1.0000

In [17]:
# Saving model
print("Saving model & model information...")
modelYML = model.to_yaml()
with open(outputDir + "model.yml", "w") as yml:
    yml.write(modelYML)
modelJSON = model.to_json()
with open(outputDir + "model.json", "w") as json:
    json.write(modelJSON)
model.save(outputDir + "model.h5")
print('Model saved! Congratulations! You finished all processes of ML!')

Saving model & model information...
Model saved! Congratulations! You finished all processes of ML!


In [18]:
# Finished